In [72]:
import numpy as np
import pandas as pd
import re
from datetime import datetime as dt
pd.options.display.max_rows = 1000

### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
ratings = pd.read_csv('ratings.csv')

In [3]:
ratings = ratings.drop(['timestamp'], axis=1)

In [4]:
def classification(row):
    if row <= 2:
        return 'низкий рейтинг'
    elif row > 2 and row <= 4:
        return 'средний рейтинг'
    elif row > 4:
        return 'высокий рейтинг'  

In [6]:
ratings['class'] = ratings['rating'].apply(classification)

In [7]:
ratings.head()

,userId,movieId,rating,class
0,1,296,5.0,высокий рейтинг
1,1,306,3.5,средний рейтинг
2,1,307,5.0,высокий рейтинг
3,1,665,5.0,высокий рейтинг
4,1,899,3.5,средний рейтинг


### Задание 2
Используем файл keywords.csv.  

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность   определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется   название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.  

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:   

geo_data = {  
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [8]:
geo_data = {
            'Центр': ['москва', 'тула', 'ярославль'],
            'Северо-Запад': ['петербург', 'псков', 'мурманск'],
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
           }

In [9]:
keywords = pd.read_csv('keywords.csv')

In [10]:
def geo_classification(row):
    for i in geo_data.keys():
        for j in geo_data[i]:
            if j[:-1:] in row['keyword']: #убираем окончание города, на тот случай 'пряники в тулЕ'
                return i
                break

In [11]:
%%time
keywords['region'] = keywords.apply(geo_classification, axis=1)

CPU times: user 5.57 s, sys: 29.6 ms, total: 5.6 s
Wall time: 5.64 s


In [12]:
keywords['region'] = keywords['region'].replace(to_replace=np.nan, value='undefined')

In [13]:
keywords['region'].unique()

array(['undefined', 'Центр', 'Северо-Запад', 'Дальний Восток'],
      dtype=object)

### Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение,   используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы    https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний   рейтинг становится ниже. 

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему   алгоритму:  

В переменную years запишите список из всех годов с 1950 по 2010.  
Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия   фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:  
для каждой строки пройдите по всем годам списка years  
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска  
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год  
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’  
Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию   рейтинга 

In [182]:
film_year = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [183]:
#убираем лишние столбцы из таблицы с названием фильмов
film_year = film_year.drop(['genres'], axis=1)
#убираем лишние столбцы из таблицы с рейтингом
ratings = ratings.drop(['userId', 'timestamp'], axis=1)

In [184]:
#находим год в названии фильмов
#так как в названии фильма может быть год, к примеру 'Бегущий по лезвию 2049 (2017)'
#берем последние значение в списке [2049, 2017]
def get_years(row):
    year = re.findall('\d{4}', row)
    if year == []:
        return [1900]
    elif len(year) > 1:
        return year[-1]
    else:
        return year

In [245]:
%%time
film_year['year'] = film_year['title'].apply(get_years)

CPU times: user 34.1 ms, sys: 70.4 ms, total: 104 ms
Wall time: 164 ms


In [246]:
#преобразуем столбец film_year['year'] в int
film_year['year'] = film_year['year'].str[0].astype(int)

In [247]:
#объединим таблицы с рейтингом и названием фильмов
film = ratings.merge(film_year, how='left', right_on='movieId', left_on='movieId')

In [248]:
#выбираем фильмы с 1950 по 2010 года
film = film.query('year >= 1950 and year <= 2010')

In [249]:
film.head()

,movieId,rating,title,year
0,296,5.0,Pulp Fiction (1994),1994.0
1,306,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994),1994.0
2,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),1993.0
3,665,5.0,Underground (1995),1995.0
4,899,3.5,Singin' in the Rain (1952),1952.0


In [250]:
film_mean_rating = film[['rating', 'year']].groupby(['year']).mean()
film_mean_rating = film_mean_rating.sort_values(['rating'], ascending=False)
film_mean_rating = film_mean_rating.reset_index()
#как мне кажется, гипотеза подтвердилась частично
#из таблицы видно, что рейтинг фильмов возрастает, относительно давности выпуска фильмов (чем старше фильм, тем он больше его рейтинг)
#но однозначно определить по году рейтинг фильма нельзя, так как
#встречаются выбросы, к примеру 1974.0	 3.931395 и 1973.0	3.769959
#то есть корреляция между годом выпуска не будет = 1
#в данном случаем было бы правильно провести дополнительные исследования и посчитать среднюю оценку за 10 лет
film_mean_rating.head(10)

,year,rating
0,1957.0,4.039520
1,1972.0,4.009470
2,1954.0,4.007824
3,1952.0,3.958236
4,1962.0,3.956259
5,1974.0,3.931395
6,1964.0,3.910245
7,1975.0,3.905185
8,1951.0,3.903898
9,1966.0,3.899744


In [251]:
#посчитаем корреляцию (хотя в данном случае это не совсем корректно, так как имеются выбросы)
#видно, что она отрицательная (то есть при росте года уменьшается рейтинг)
#но при этом, коэфф корреляции средний
film_mean_rating['rating'].corr(film_mean_rating['year'])

-0.8189896792159558